In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the TSV file
df = pd.read_csv('ml-32m/ratings.csv')


# Print the head of the DataFrame
print(df.head())

/Users/alexanderaujesky/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40


In [2]:
data_folder = './MovieSummaries/MovieSummaries/'
#paths to files
plot_summaries_path = data_folder + 'plot_summaries.txt'
movie_metadata_path = data_folder + 'movie.metadata.tsv'
character_metadata_path = data_folder + 'character.metadata.tsv'

# load the data
# 1. Plot summaries data
plot_summaries_df = pd.read_csv(plot_summaries_path, delimiter='\t', names=['wikipedia_movie_id', 'plot_summary'], 
                                 encoding='utf-8')

# 2. Movie metadata
movie_metadata_df = pd.read_csv(movie_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 
                                                                            'movie_name', 'release_date', 'box_office_revenue',
                                                                            'runtime', 'languages', 'countries', 'genres'], 
                                 encoding='utf-8')

# 3. Character metadata
character_metadata_df = pd.read_csv(character_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 
                                                                                    'actor_dob', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 
                                                                                    'actor_age_at_release', 'freebase_character_actor_map_id', 'freebase_character_id', 
                                                                                    'freebase_actor_id'], 
                                     encoding='utf-8')


In [3]:
movies = pd.read_csv('ml-32m/movies.csv')

In [4]:
movie_metadata_df['movie_name_formatted'] = movie_metadata_df['movie_name'].str.lower().str.strip()
movies['title_format'] = movies['title'].str[:-6].str.strip().str.lower()


In [5]:
movies.head()

,movieId,title,genres,title_format
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii


In [6]:
# calculate the number of common movies between the two datasets

common_movies = set(movie_metadata_df['movie_name_formatted']).intersection(set(movies['title_format']))
print('Number of common movies:', len(common_movies))

# merge the two datasets

merged_df = pd.merge(movies, movie_metadata_df, left_on='title_format', right_on='movie_name_formatted', how='inner')

Number of common movies: 12318


In [7]:
merged_df

,movieId,title,genres_x,title_format,wikipedia_movie_id,freebase_movie_id,movie_name,release_date,box_office_revenue,runtime,languages,countries,genres_y,movie_name_formatted
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,53085,/m/0dyb1,Toy Story,1995-11-19,361958736.0,77.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0556j8"": ""Buddy film"", ""/m/03k9fj"": ""Adve...",toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji,3700174,/m/09w353,Jumanji,1995-12-15,262797249.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/0hj3n2k"": ""Fanta...",jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men,1934035,/m/0676dr,Grumpier Old Men,1995-12-22,71518503.0,101.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0556j8"": ""...",grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale,972970,/m/03vny7,Waiting to Exhale,1995-12-22,81452156.0,121.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3n0w"": ""Ensemble Film"", ""/m/06w2n3t"": ...",waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii,3303622,/m/094g2z,Father of the Bride Part II,1995-12-08,76594107.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06qm3"": ""Screwball comedy"", ""/m/02l7c8"": ...",father of the bride part ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,131152,The Fat Spy (1966),Comedy,the fat spy,1853200,/m/0613sx,The Fat Spy,1966,NaN,75.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0gf28"": ""Parody"", ""/m/0220p9g"": ""Musical ...",the fat spy
17558,131166,WWII IN HD (2009),(no genres listed),wwii in hd,26321497,/m/0bbxzy2,WWII in HD,NaN,NaN,NaN,{},{},"{""/m/082gq"": ""War film""}",wwii in hd
17559,131239,Three Quarter Moon (2011),Comedy|Drama,three quarter moon,33597212,/m/0hgrd5g,Three Quarter Moon,2011-09-30,NaN,95.0,"{""/m/02hwyss"": ""Turkish Language"", ""/m/0gtg"": ...","{""/m/0345h"": ""Germany""}","{""/m/0556j8"": ""Buddy film"", ""/m/0hqxf"": ""Famil...",three quarter moon
17560,131248,Brother Bear 2 (2006),Adventure|Animation|Children|Comedy|Fantasy,brother bear 2,4118248,/m/0bk2k2,Brother Bear 2,2006-08-29,NaN,73.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hqxf"": ""Family Film"", ""/m/0hcr"": ""Animat...",brother bear 2


In [8]:
# Remove this user from the dataset, because it has too many ratings
df = df[df['userId'] != 175325]

In [10]:
list_of_genres=movies['genres'].unique()

In [11]:
# Print the user with the most ratings
user_ratings_count = df['userId'].value_counts()

print('User with the most ratings:')
print(user_ratings_count.head(20))


User with the most ratings:
118205    9254
8405      7515
82418     5646
121535    5520
125794    5491
74142     5447
34576     5356
131904    5330
83090     5169
59477     4988
130767    4785
79159     4707
8963      4524
15617     4354
92011     4236
71975     4182
20132     4101
46470     4094
88820     4093
63147     3958
Name: userId, dtype: int64


In [29]:
rating_films=pd.merge(movies,df,on='movieId',how='inner')


In [28]:
unique_genres = movies['genres'].unique()
all_genres = set('|'.join(unique_genres).split('|'))

all_genres_list = list(all_genres)
all_genres_list.remove('(no genres listed)')
print(all_genres_list)


['Comedy', 'Documentary', 'Children', 'Crime', 'Animation', 'Romance', 'Film-Noir', 'Drama', 'IMAX', 'Thriller', 'Horror', 'Adventure', 'Sci-Fi', 'Western', 'War', 'Action', 'Musical', 'Fantasy', 'Mystery']


In [36]:
for genre in all_genres_list : 
    rating_films[genre] = rating_films['genres'].apply(lambda x: genre in x.split('|'))
rating_films

,movieId,title,genres,title_format,userId,rating,timestamp,Comedy,Documentary,Children,...,Thriller,Horror,Adventure,Sci-Fi,Western,War,Action,Musical,Fantasy,Mystery
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,3,4.0,1999-12-11 13:36:47,True,False,True,...,False,False,True,False,False,False,False,False,True,False
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,6,5.0,1997-03-13 17:50:52,True,False,True,...,False,False,True,False,False,False,False,False,True,False
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,8,4.0,1996-06-05 13:37:51,True,False,True,...,False,False,True,False,False,False,False,False,True,False
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,10,4.0,1999-11-25 02:44:47,True,False,True,...,False,False,True,False,False,False,False,False,True,False
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,11,4.5,2009-01-02 01:13:41,True,False,True,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben (2007),Comedy,kein bund für's leben,79570,4.0,2015-03-30 19:32:59,True,False,False,...,False,False,False,False,False,False,False,False,False,False
20000259,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,"feuer, eis & dosenbier",79570,4.0,2015-03-30 19:48:08,True,False,False,...,False,False,False,False,False,False,False,False,False,False
20000260,131258,The Pirates (2014),Adventure,the pirates,28906,2.5,2015-03-30 19:56:32,False,False,False,...,False,False,True,False,False,False,False,False,False,False
20000261,131260,Rentun Ruusu (2001),(no genres listed),rentun ruusu,65409,3.0,2015-03-30 19:57:46,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [73]:
def rating_for_userID_generator(userId):
    user_rating = rating_films[rating_films['userId'] == userId]
    user_rating = user_rating.iloc[:, 5:].sample(frac=1)  
    user_rating = user_rating.drop(columns=['timestamp'])
    y = user_rating['rating']
    x = user_rating.drop(columns=['rating'])
    
    length = len(y)
    x_size = int(length * 0.8) 
    
    x_train = x.iloc[:x_size, :]
    x_test = x.iloc[x_size:, :]
    y_train = y.iloc[:x_size]
    y_test = y.iloc[x_size:]
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = rating_for_userID_generator(8405)


In [75]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score

# Define the neural network model
model = Sequential([
    Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Predict on both training and test sets
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Evaluate the model
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Training Set Evaluation:")
print("MSE:", train_mse)
print("R^2 Score:", train_r2)

print("\nTest Set Evaluation:")
print("MSE:", test_mse)
print("R^2 Score:", test_r2)



2024-11-12 14:47:01.392134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
151/151 [==============================] - 2s 4ms/step - loss: 2.8266 - mse: 2.8266 - val_loss: 0.9593 - val_mse: 0.9593
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.8281 - mse: 0.8281 - val_loss: 0.8343 - val_mse: 0.8343
Epoch 3/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7768 - mse: 0.7768 - val_loss: 0.8241 - val_mse: 0.8241
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7608 - mse: 0.7608 - val_loss: 0.7988 - val_mse: 0.7988
Epoch 5/100
151/151 [==============================] - 0s 3ms/step - loss: 0.7497 - mse: 0.7497 - val_loss: 0.7973 - val_mse: 0.7973
Epoch 6/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7420 - mse: 0.7420 - val_loss: 0.7924 - val_mse: 0.7924
Epoch 7/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7395 - mse: 0.7395 - val_loss: 0.7937 - val_mse: 0.7937
Epoch 8/100
151/151 [==============================] - 0s 2ms/step - 